In [1]:
import numpy as np
import random

class AGMOEA:
    def __init__(self, NP, K, NGBA, NEXA, Tmax, n):
        self.NP = NP
        self.K = K
        self.NGBA = NGBA
        self.NEXA = NEXA
        self.Tmax = Tmax # The maximum generations
        self.EXA = []  # External archive
        self.GBA = [[] for _ in range(K)]  # Subspace grid
        self.current_generation = 0
        self.Pm = 1/n # mutation probability
        self.n = n # number of decision variables
        self.S = set()
        self.operators = ['BLX-α', 'SBX', 'SPX', 'PCX', 'DE/rand/1']
        # there is a bug here
        self.operator_usage = {operator: 0 for operator in self.operators}
        self.operator_probabilities = {operator: 1.0 / len(self.operators) for operator in self.operators}
        self.pmin = 0.1  # Minimum selection probability for each operator


    def initialize_parameters(self):
        # Initialize parameters such as population size (NP), interval number of each objective (K),
        # maximum size of each subspace (NGA), and maximum size of the external archive (NEXA)
        pass

    def construct_subspaces(self, solutions):
        # Calculate the grid intervals based on the ideal and nadir points
        grid_intervals = (self.nadir_point - self.ideal_point) / self.K

        # Something is wrong here!!!!
        # Reset the grid for the new generation
        self.GBA = [Subspace(np.full_like(self.ideal_point, i)) for i in range(self.K)]

        # Assign each solution to the appropriate subspace
        for solution in solutions:
            # Calculate the relative position of the solution in the objective space
            relative_position = solution.objective_values - self.ideal_point
            # Determine the grid coordinates
            grid_coordinates = np.floor(relative_position / grid_intervals).astype(int)
            # Find the corresponding subspace and add the solution to it
            self.GBA[tuple(grid_coordinates)].solutions.append(solution)


    def improve_EXA(self):
        # Improve the external archive EXA using the extension mechanism described in Section 3.6
        pass

    def SR(self, subspace):
        """
        Calculate the Subspace Ranking (SR) based on the summation of its coordinates.
        :param subspace: A Subspace instance.
        :return: The SR value as the sum of the subspace's coordinates.
        """
        # Assuming subspace.grid_coordinates is a list or array of coordinates
        return sum(subspace.grid_coordinates)

  
    
    def select_subspace(self):
        """
        Selects a subspace with an adaptive strategy.

        :return: The selected subspace.
        """
        epsilon = 1e-6  # A small constant to prevent division by zero
        G_minus_S = [subspace for subspace in self.GBA if subspace not in self.S]  # Exclude degraded subspaces

        probabilities = {k: (1 / (self.SR(k) + epsilon)) for k in G_minus_S}
        total = sum(probabilities.values())
        normalized_probabilities = {k: (v / total) for k, v in probabilities.items()}

        # Select a subspace based on the calculated probabilities
        selected_subspace = random.choices(list(normalized_probabilities.keys()), weights=normalized_probabilities.values(), k=1)[0]

        # Update the set of degraded subspaces based on the selection
        self.update_degraded_subspaces(selected_subspace)

        return selected_subspace

    def update_degraded_subspaces(self, selected_subspace):
        """
        Update the set of degraded subspaces based on the selected subspace.

        :param selected_subspace: The subspace that was selected.
        """
        for subspace in self.GBA:
            if selected_subspace.strong_subspace_dominance(subspace):
                self.S.add(subspace)


    def adaptive_selection_probability(self):
        pre = 0.8
        pRE = pre / (1.0 + np.exp(-20 * (self.current_generation / self.Tmax - 0.25)))
        return pRE
    
    def parent_selection(self, selected_subspace):
        """
        Selects a parent from the selected subspace based on an adaptive strategy.

        :param selected_subspace: The subspace from which to select a parent.
        :return: The selected parent individual.
        """
        if random.random() < self.adaptive_selection_probability():
            # Select a representative individual based on some criteria
            parent1 = selected_subspace.select_representative()
            if parent1 is None:
                parent1 = random.choice(self.EXA)
        else:
            # Select a random individual from the solutions within the selected subspace
            parent1 = random.choice(selected_subspace.solutions)
        parent2 = random.choice(self.EXA)
        # in case some operators need 3 parents
        parent3 = random.choice(self.EXA)
        return parent1, parent2, parent3

    def update_operator_probabilities(self):
        total_solutions = len(self.EXA)
        if total_solutions > 0:
            # Calculate new probabilities based on usage
            for operator in self.operators:
                # Crossover probability is based on number of solutions they generated and is in EXA
                # there is a bug here
                self.operator_probabilities[operator] = max(self.operator_usage[operator] / total_solutions, self.pmin)

            # Normalize probabilities if they exceed 1 after enforcing pmin
            total_probability = sum(self.operator_probabilities.values())
            if total_probability > 1.0:
                for operator in self.operators:
                    self.operator_probabilities[operator] /= total_probability

    def select_crossover_operator(self):
        # Perform roulette-wheel selection
        operators, probabilities = zip(*self.operator_probabilities.items())
        selected_operator = random.choices(operators, weights=probabilities, k=1)[0]
        return selected_operator

    def generate_offspring(self, parent1, parent2):
        # Select a crossover operator based on updated probabilities
        selected_operator = self.select_crossover_operator()

        # Use the selected operator to perform crossover and generate offspring
        # This is a placeholder for the actual crossover implementation
        offspring = crossover(selected_operator, parent1, parent2)

        # Update usage count for the selected operator
        self.operator_usage[selected_operator] += 1

        return offspring

    def evaluate_individual(self, individual):
        # Evaluate the fitness of an individual
        pass

    def fast_non_dominated_sort(self, population):
        # Sort the population based on non-domination criteria
        pass
    def crowding_distance(self):
        pass

    def agmoea_algorithm(self):
        # Main procedure of the AGMOEA algorithm
        self.initialize_parameters()

        # Generate initial population P0
        P0 = []  # Initial population

        # Evaluate individuals in P0
        for individual in P0:
            self.evaluate_individual(individual)

        # Store non-dominated solutions in P0 into EXA
        self.EXA.extend(self.fast_non_dominated_sort(P0))

        # Main loop
        while not self.termination_criterion():
            self.S.clear()
            # Construct subspaces
            self.construct_subspaces()

            # Improve EXA
            self.improve_EXA()

            # Set TP to be empty
            TP = []  # Temporary population

            # Generate offsprings and evaluate
            for _ in range(NP):
                # Select a subspace
                selected_subspace, S = self.select_subspace(G, S)
                
                # Generate an offspring
                offspring = self.generate_offspring(subspace)
                
                # Evaluate offspring and store in TP
                self.evaluate_individual(offspring)
                TP.append(offspring)

            # Update EXA with TP
            self.EXA.extend(self.fast_non_dominated_sort(TP))

            # Update population P with TP
            P = TP

            # Set P to be the set of the best NP individuals in P based on fast non-dominated sorting
            P = self.fast_non_dominated_sort(P)[:NP]

    def termination_criterion(self):
        # Define the termination criterion for the algorithm
        pass

